# Abstract
Abstract (Work in progress):

Flight delays are a costly and frustrating inevitability for even the savviest of flyers. Itineraries that accurately account for delays stand to ease a major pain point for travelers on commercial airlines. Our project seeks to address this problem by predicting the length of flight delays using machine learning. To this end, we propose the use of a supervised Support Vector Regression model. Our proposed dataset contains flight data from 2009 to 2019, gathered from the US Department of Transportation's Bureau of Transportation Statistics. We plan to implement a machine learning pipeline for training and testing. The output of our final model will enable us to identify the variables associated with longer delays.

# Datasets

**Dataset #1**: Airline Delay Analysis - contains relevant information about most US flights for a period of ten years beginning in 2009 and ending in 2019.

Link: https://www.kaggle.com/datasets/sherrytp/airline-delay-analysis

Location: /content/airline_delay_analysis


# Team

Jason Bulux
jbulux@ucsd.edu

Rosario Ortiz
mdortiz@ucsd.edu

Kwadwo Danquah
kdanquah@ucsd.edu

Sam Kogan
skogan@ucsd.edu